In [2]:
import pandas as pd

In [6]:
with pd.ExcelFile('PD 2021 Wk 4 Input.xlsx') as xl:
    df_sales_uncleaned = pd.concat([pd.read_excel(xl, s).assign(Store=s) for s in xl.sheet_names if 'Targets' not in s])

In [7]:
# pivot the table and split
df_cleaned = df_sales_uncleaned.melt(id_vars=['Date', 'Store'], value_vars=['New - Saddles', 'New - Mudguards', 'New - Wheels', 'New - Bags', 'Existing - Saddles', 'Existing - Mudguards', 'Existing - Wheels', 'Existing - Bags'], value_name='Sales Amount', var_name='Type')
df_cleaned[['Customer Type', 'Product']] = df_cleaned['Type'].str.split(' - ', expand=True)
df_cleaned.drop(columns='Type', inplace=True)

# get the quarter
df_cleaned['Quarter'] = df_cleaned['Date'].dt.quarter

df_cleaned

,Date,Store,Sales Amount,Customer Type,Product,Quarter
0,2021-01-21,Manchester,13,New,Saddles,1
1,2021-02-21,Manchester,1,New,Saddles,1
2,2021-03-21,Manchester,8,New,Saddles,1
3,2021-04-21,Manchester,3,New,Saddles,2
4,2021-05-21,Manchester,2,New,Saddles,2
...,...,...,...,...,...,...
475,2021-08-21,Birmingham,11,Existing,Bags,3
476,2021-09-21,Birmingham,24,Existing,Bags,3
477,2021-10-21,Birmingham,16,Existing,Bags,4
478,2021-11-21,Birmingham,12,Existing,Bags,4


In [13]:
# group the table by quarter to prepare it for joining
df_cleaned_per_quarter = df_cleaned.groupby(['Store', 'Quarter'], as_index=False)['Sales Amount'].sum()
df_cleaned_per_quarter

# load the targets table and join it
df_targets = pd.read_excel('PD 2021 Wk 4 Input.xlsx', sheet_name='Targets')

# join
df_output = pd.merge(df_cleaned_per_quarter, df_targets, how='left', on=['Store', 'Quarter'])

# get variance to target
df_output['Variance to Target'] = df_output['Sales Amount'] - df_output['Target']
df_output

,Store,Quarter,Sales Amount,Target,Variance to Target
0,Birmingham,1,477,475,2
1,Birmingham,2,346,325,21
2,Birmingham,3,348,300,48
3,Birmingham,4,404,400,4
4,Leeds,1,488,490,-2
5,Leeds,2,331,325,6
6,Leeds,3,279,300,-21
7,Leeds,4,349,400,-51
8,London,1,425,475,-50
9,London,2,324,325,-1


In [15]:
# get the rank per quarter
df_output['rank'] = df_output.groupby('Quarter')['Variance to Target'].rank()
df_output.sort_values('Quarter')

,Store,Quarter,Sales Amount,Target,Variance to Target,rank
0,Birmingham,1,477,475,2,4.0
16,York,1,499,490,9,5.0
4,Leeds,1,488,490,-2,3.0
12,Manchester,1,440,475,-35,2.0
8,London,1,425,475,-50,1.0
17,York,2,329,300,29,5.0
13,Manchester,2,288,300,-12,1.0
9,London,2,324,325,-1,2.0
5,Leeds,2,331,325,6,3.0
1,Birmingham,2,346,325,21,4.0
